<a href="https://colab.research.google.com/github/vfrantc/weather_experiments/blob/main/001_quaternion_decomposition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Wed Jul 27 16:39:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    42W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install python-box
!pip install kornia
!pip install thop
!pip install onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.0 MB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 517 kB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.1 MB 3.8 MB/s 


In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
!git clone https://github.com/vfrantc/deweather.git

Cloning into 'deweather'...
remote: Enumerating objects: 649, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 649 (delta 50), reused 167 (delta 34), pack-reused 456
Receiving objects: 100% (649/649), 23.86 MiB | 43.24 MiB/s, done.
Resolving deltas: 100% (309/309), done.


In [ ]:
%cd deweather/QTransWeather-dehaze

/content/deweather/QTransWeather-dehaze


In [ ]:
!cp /content/drive/MyDrive/deweather2/split_star.zip .
!unzip split_star.zip

Archive:  split_star.zip
   creating: input/
  inflating: input/parking_lot_10997.png  
  inflating: input/348.png           
  inflating: input/parking_lot_01124.png  
  inflating: input/parking_lot_09578.png  
  inflating: input/412.png           
  inflating: input/374.png           
  inflating: input/360.png           
  inflating: input/406.png           
  inflating: input/im_0493_s95_a05.png  
  inflating: input/im_0493_s85_a05.png  
  inflating: input/17_rain.png       
  inflating: input/638.png           
  inflating: input/parking_lot_09008.png  
  inflating: input/16_rain.png       
  inflating: input/parking_lot_00004.png  
  inflating: input/176.png           
  inflating: input/610.png           
  inflating: input/604.png           
  inflating: input/162.png           
  inflating: input/parking_lot_00951.png  
  inflating: input/parking_lot_09785.png  
  inflating: input/parking_lot_09791.png  
  inflating: input/parking_lot_10439.png  
  inflating: input/parking_lot

In [ ]:
import sys
sys.path.insert(0,'/content/deweather/QTransWeather-dehaze/')

import os
import time
import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.models import vgg16
import kornia as K

import cv2
import numpy as np
from tqdm.notebook import tqdm
from glob import glob
from PIL import Image
from box import Box

from qcnn import QuaternionConv
from qcnn import get_r, get_i, get_j, get_k

In [ ]:
opt = Box({'epochs': 100,
           'batch_size': 16,
           'patch_size': 128,
           'lr': 0.001})

In [ ]:
# --- Perceptual loss network  --- #
class LossNetwork(torch.nn.Module):
    def __init__(self, vgg_model):
        super(LossNetwork, self).__init__()
        self.vgg_layers = vgg_model
        self.layer_name_mapping = {
            '3': "relu1_2",
            '8': "relu2_2",
            '15': "relu3_3"
        }

    def output_features(self, x):
        output = {}
        for name, module in self.vgg_layers._modules.items():
            x = module(x)
            if name in self.layer_name_mapping:
                output[self.layer_name_mapping[name]] = x
        return list(output.values())

    def forward(self, pred_im, gt):
        loss = []
        pred_im_features = self.output_features(pred_im)
        gt_features = self.output_features(gt)
        for pred_im_feature, gt_feature in zip(pred_im_features, gt_features):
            loss.append(F.mse_loss(pred_im_feature, gt_feature))

        return sum(loss)/len(loss)

vgg_model = vgg16(pretrained=True).features[:16]
vgg_model = vgg_model.cuda()
# vgg_model = nn.DataParallel(vgg_model, device_ids=device_ids)
for param in vgg_model.parameters():
    param.requires_grad = False
loss_network = LossNetwork(vgg_model)
loss_network.eval()

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


LossNetwork(
  (vgg_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
  )
)

In [106]:
class DecomNet(nn.Module):
    def __init__(self, channel=64, kernel_size=3):
        super(DecomNet, self).__init__()
        # Shallow feature extraction
        self.net1_conv0 = QuaternionConv(8, channel, kernel_size * 3, stride=1, padding=4)
        # Activated layers!
        self.net1_convs = nn.Sequential(QuaternionConv(channel, channel, kernel_size, stride=1, padding=1),
                                        nn.ReLU(),
                                        QuaternionConv(channel, channel, kernel_size, stride=1, padding=1),
                                        nn.ReLU(),
                                        QuaternionConv(channel, channel, kernel_size, stride=1, padding=1),
                                        nn.ReLU(),
                                        QuaternionConv(channel, channel, kernel_size, stride=1, padding=1),
                                        nn.ReLU(),
                                        QuaternionConv(channel, channel, kernel_size, stride=1, padding=1),
                                        nn.ReLU())
        # Final recon layer
        self.net1_recon = QuaternionConv(channel, 8, kernel_size, stride=1, padding=1)

    def edge_compute(self, x):
        x_diffx = torch.abs(x[:,:,:,1:] - x[:,:,:,:-1])
        x_diffy = torch.abs(x[:,:,1:,:] - x[:,:,:-1,:])

        y = x.new(x.size())
        y.fill_(0)
        y[:,:,:,1:] += x_diffx
        y[:,:,:,:-1] += x_diffx
        y[:,:,1:,:] += x_diffy
        y[:,:,:-1,:] += x_diffy
        #y = torch.sum(y,1,keepdim=True)/3
        y /= 4
        return y

    def forward(self, input_im):
        b, c, h, w = input_im.shape
        real = torch.zeros((b, 1, h, w)).cuda()

        edges = self.edge_compute(input_im)
        edges = torch.cat((real, edges), dim=1)
        input_im = torch.cat((real, input_im), dim=1)

        input = torch.cat((get_r(edges), get_r(input_im), get_i(edges), get_i(input_im), get_j(edges), get_j(input_im), get_k(edges), get_k(input_im)), dim=1)
        feats0   = self.net1_conv0(input)
        featss   = self.net1_convs(feats0)
        outs     = self.net1_recon(featss)

        b = outs[:, 0, :, :].unsqueeze(1)
        R        = torch.sigmoid(torch.cat((outs[:, 0, :, :].unsqueeze(1), outs[:, 2, :, :].unsqueeze(1), outs[:, 4, :, :].unsqueeze(1), outs[:, 6, :, :].unsqueeze(1)), dim=1))
        L        = torch.sigmoid(torch.cat((outs[:, 1, :, :].unsqueeze(1), outs[:, 3, :, :].unsqueeze(1), outs[:, 5, :, :].unsqueeze(1), outs[:, 7, :, :].unsqueeze(1)), dim=1))
        return R, L

In [107]:
net = DecomNet()
net = net.cuda()
lr = opt.lr * np.ones([opt.epochs])
lr[20:] = lr[0] / 10.0

train_input_data_names = glob('./input/*.png')
train_input_data_names.sort()
train_slow_data_names = glob('./slow/*.png')
train_slow_data_names.sort()
train_fast_data_names = glob('./fast/*.png')
train_fast_data_names.sort()

In [108]:
train_op = optim.Adam(net.parameters(), lr=lr[0], betas=(0.9, 0.999))

In [ ]:
image_id   = 0
numBatch = len(train_input_data_names) // int(opt.batch_size)
start_time = time.time()
epoch = 0
iter_num = 0
for epoch in range(0, opt.epochs):
    slr = lr[epoch]

    # Adjust learning rate
    for param_group in train_op.param_groups:
        param_group['lr'] = slr

    for batch_id in range(0, numBatch):
        # Generate training data for a batch
        batch_input = np.zeros((opt.batch_size, 3, opt.patch_size, opt.patch_size,), dtype="float32")
        batch_slow = np.zeros((opt.batch_size, 3, opt.patch_size, opt.patch_size,), dtype="float32")
        batch_fast = np.zeros((opt.batch_size, 3, opt.patch_size, opt.patch_size,), dtype="float32")

        for patch_id in range(opt.batch_size):
            # Load images
            train_input_img = Image.open(train_input_data_names[image_id])
            train_input_img = 2*np.array(train_input_img, dtype='float32')/255.0 - 1
            train_slow_img= Image.open(train_slow_data_names[image_id])
            train_slow_img= np.array(train_slow_img, dtype='float32')/255.0
            train_fast_img= Image.open(train_fast_data_names[image_id])
            train_fast_img= np.array(train_fast_img, dtype='float32')/255.0

            # Take random crops
            h, w, _        = train_input_img.shape
            x = random.randint(0, h - opt.patch_size)
            y = random.randint(0, w - opt.patch_size)

            train_input_img = train_input_img[x: x + opt.patch_size, y: y + opt.patch_size, :]
            train_slow_img= train_slow_img[x: x + opt.patch_size, y: y + opt.patch_size, :]
            train_fast_img= train_fast_img[x: x + opt.patch_size, y: y + opt.patch_size, :]

            # Data augmentation
            if random.random() < 0.5:
                train_input_img = np.flipud(train_input_img)
                train_slow_img= np.flipud(train_slow_img)
                train_fast_img= np.flipud(train_fast_img)
            if random.random() < 0.5:
                train_input_img = np.fliplr(train_input_img)
                train_slow_img= np.fliplr(train_slow_img)
                train_fast_img= np.fliplr(train_fast_img)
            rot_type = random.randint(1, 4)
            if random.random() < 0.5:
                train_input_img = np.rot90(train_input_img, rot_type)
                train_slow_img= np.rot90(train_slow_img, rot_type)
                train_fast_img= np.rot90(train_fast_img, rot_type)
            
            # Permute the images to tensor format
            train_input_img = np.transpose(train_input_img, (2, 0, 1))
            train_slow_img= np.transpose(train_slow_img, (2, 0, 1))
            train_fast_img= np.transpose(train_fast_img, (2, 0, 1))
            
            # Prepare the batch
            batch_input[patch_id, :, :, :] = train_input_img
            batch_slow[patch_id, :, :, :]= train_slow_img
            batch_fast[patch_id, :, :, :]= train_fast_img

            image_id = (image_id + 1) % len(train_input_data_names)
            if image_id == 0:
                tmp = list(zip(train_input_data_names, train_slow_data_names, train_fast_data_names))
                random.shuffle(list(tmp))
                train_input_data_names, train_slow_data_names, train_fast_data_names = zip(*tmp)

        input = Variable(torch.FloatTensor(torch.from_numpy(batch_input))).cuda()
        b, c, h, w = input.shape
        real = torch.zeros((b, 1, h, w)).cuda()
        target_slow = Variable(torch.FloatTensor(torch.from_numpy(batch_slow))).cuda()
        target_slow = torch.cat((real, target_slow), dim=1)
        input_target = torch.cat((real, input), dim=1)

        target_fast = Variable(torch.FloatTensor(torch.from_numpy(batch_fast))).cuda()
        target_fast = torch.cat((real, target_fast), dim=1)

        out_fast, out_slow = net(input)
        train_op.zero_grad()
        loss = F.l1_loss(out_fast*out_slow,  (input_target + 1)/2) \
             + F.l1_loss(out_slow, target_slow) \
             + F.l1_loss(out_fast, target_fast) \
             + 0.04*loss_network(out_slow[:, 1:, :, :], target_slow[:, 1:, :, :]) \
             + 0.1*loss_network(out_fast[:, 1:, :, :], target_fast[:, 1:, :, :])

        loss.backward()
        train_op.step()

        print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.6f" % (epoch + 1, batch_id + 1, numBatch, time.time() - start_time, loss.item()))
        iter_num += 1

print("Finished training...")

Epoch: [ 1] [   1/  70] time: 0.6183, loss: 1.095553
Epoch: [ 1] [   2/  70] time: 1.2397, loss: 1.079693
Epoch: [ 1] [   3/  70] time: 1.8562, loss: 1.033405
Epoch: [ 1] [   4/  70] time: 2.4901, loss: 0.917969
Epoch: [ 1] [   5/  70] time: 3.1129, loss: 0.799846
Epoch: [ 1] [   6/  70] time: 3.7283, loss: 0.629007
Epoch: [ 1] [   7/  70] time: 4.3637, loss: 0.609332
Epoch: [ 1] [   8/  70] time: 4.9900, loss: 0.462295
Epoch: [ 1] [   9/  70] time: 5.5928, loss: 0.445799
Epoch: [ 1] [  10/  70] time: 6.2397, loss: 0.494335
Epoch: [ 1] [  11/  70] time: 6.8765, loss: 0.436891
Epoch: [ 1] [  12/  70] time: 7.4731, loss: 0.455273
Epoch: [ 1] [  13/  70] time: 8.0650, loss: 0.356756
Epoch: [ 1] [  14/  70] time: 8.6823, loss: 0.434996
Epoch: [ 1] [  15/  70] time: 9.2879, loss: 0.464784
Epoch: [ 1] [  16/  70] time: 9.8537, loss: 0.426411
Epoch: [ 1] [  17/  70] time: 10.4746, loss: 0.415591
Epoch: [ 1] [  18/  70] time: 11.0988, loss: 0.415857
Epoch: [ 1] [  19/  70] time: 11.7332, loss:

In [ ]:
torch.save(net.state_dict(), 'qdecomp.tar')
!cp qdecomp.tar /content/drive/MyDrive/deweather2

In [ ]:
def get_decom(trainable=True):
    net = DecomNet().cuda()
    ckpt_dict  = torch.load('qdecomp.tar') # , map_location=torch.device('cpu')
    net.load_state_dict(ckpt_dict)
    for p in net.parameters():
        p.requires_grad = trainable
    return net

In [ ]:
def decom_image(image, net):
    test_low_img   = 2*cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0 - 1
    test_low_img   = np.transpose(test_low_img, (2, 0, 1))
    input_low_test = np.expand_dims(test_low_img, axis=0)
    input_low_test = Variable(torch.FloatTensor(torch.from_numpy(input_low_test))).cuda()
    R_low, I_low   = net(input_low_test)
    R_low = np.clip(np.transpose(R_low.cpu().detach().numpy().squeeze(), (1, 2, 0)), 0, 1)
    I_low = np.clip(np.transpose(I_low.cpu().detach().numpy().squeeze(), (1, 2, 0)), 0, 1)
    return R_low[:, :, 1:], I_low[:, :, 1:]

In [ ]:
!cp /content/drive/MyDrive/deweather2/input.zip .
!unzip input.zip 

Archive:  input.zip
  inflating: input/get_list.py.ipynb  
  inflating: input/.DS_Store         
  inflating: __MACOSX/input/._.DS_Store  
  inflating: input/natural.txt       
   creating: input/input/
   creating: input/.ipynb_checkpoints/
  inflating: input/input/348.png     
  inflating: input/input/412.png     
  inflating: input/input/374.png     
  inflating: input/input/360.png     
  inflating: input/input/406.png     
  inflating: input/input/638.png     
  inflating: input/input/176.png     
  inflating: input/input/610.png     
  inflating: input/input/604.png     
  inflating: input/input/162.png     
  inflating: input/input/002.png     
  inflating: input/input/016.png     
  inflating: input/input/228.png     
  inflating: input/input/566.png     
  inflating: input/input/200.png     
  inflating: input/input/214.png     
  inflating: input/input/572.png     
  inflating: input/input/599.png     
  inflating: input/input/215.png     
  inflating: input/input/573.png    

In [ ]:
net = get_decom(trainable=False)

In [ ]:
import matplotlib.pyplot as plt 
FNAME = 'input/input/001.png'
dehazed_image = cv2.imread(FNAME)
reflectance, illumination = decom_image(dehazed_image, net)


fig, axs = plt.subplots(2, figsize=(12, 16))
axs[0].imshow(reflectance)
axs[1].imshow(illumination, cmap='gray')

plt.figure()
plt.imshow(reflectance*illumination)
plt.show()